# Evaluating a Traceable Function

If you are tracing some application using LangSmith's `@traceable` decorator, the `run_on_dataset` function should
automatically configure it to include all runs within the appropriate trace if it is directly passed in to the function.

This functionality is supported in `langchain>=0.0.323`.

In [1]:
import openai
from langchain.schema.runnable import RunnableLambda
from langsmith import traceable


@traceable(run_type="llm")
def complete(*args, **kwargs):
    return openai.ChatCompletion.create(
        *args,
        **kwargs,
    )


@traceable()
def my_model(question: str, context: str):
    messages = [
        {
            "role": "system",
            "content": f"Respond to the user, taking into account the context: {context}",
        },
        {
            "role": "user",
            "content": question,
        },
    ]
    completion = complete(model="gpt-3.5-turbo", messages=messages)
    # It should still work if a LangChain component is called within a traceable function
    format = RunnableLambda(lambda x: x.choices[0].message.content)
    return format.invoke(completion)

In [2]:
import uuid

from langsmith import Client

client = Client()
uid = uuid.uuid4()
examples = [
    {
        "question": "What's 2+2",
        "context": "You are a pirate",
    },
    {
        "question": "Where did coffee originate from?",
        "context": "You are a knight of King Arthur.",
    },
]
dataset_name = f"Evaluating Traceables Walkthrough - {uid}"
dataset = client.create_dataset(dataset_name)

client.create_examples(
    inputs=examples,
    dataset_id=dataset.id,
)

In [3]:
from langchain.smith import RunEvalConfig

evaluation = RunEvalConfig(
    evaluators=["criteria"],
)

client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=my_model,
    evaluation=evaluation,
)

View the evaluation results for project 'test-abandoned-blossom-92' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/e11bc750-2e63-4109-907f-61dd905588fa?eval=true
[------------------------------------------------->] 2/2

{'project_name': 'test-abandoned-blossom-92',
 'results': {'03ae9ac9-161a-4eea-9446-a7ca5562b96f': {'output': 'My liege, coffee is said to have originated in the ancient land of Ethiopia. Legend has it that a goat herder named Kaldi discovered the energizing effects of coffee beans after noticing his goats became lively and energetic upon consuming them. This marvelous drink eventually made its way to the Arabian Peninsula and beyond, becoming a cherished beverage throughout the world.',
   'input': {'context': 'You are a knight of King Arthur.',
    'question': 'Where did coffee originate from?'},
   'feedback': []},
  '508cf84a-af98-4283-bcd5-e2bf59801f99': {'output': "Ahoy, matey! The answer be 4! If ye be lookin' for a challenge, I can teach ye the ways of navigation or the art of swashbucklin' instead!",
   'input': {'context': 'You are a pirate', 'question': "What's 2+2"},
   'feedback': []}}}

## Promoting to LangChain component

In general, it is recommended to use LangChain runnables throughout your LLM application wherever you want tracing and callbacks. Beta support for composing `@traceable` functions with other LangChain runnables is provided via the `RunnableTraceable` class.

In [23]:
from langchain.chat_models import ChatOpenAI
from langsmith.run_helpers import as_runnable

chain = (
    as_runnable(my_model) | ChatOpenAI()
)  # the second call will respond to the first

In [24]:
chain.invoke(
    {
        "question": "Why is blue brown?",
        "context": "You are a pirate",
    }
)

AIMessage(content="Ahoy there, me hearty! Ye be askin' a question about the colors of the sea, aye? Well, let me tell ye a tale. The sea be a mighty and mysterious place, filled with wonders beyond our reckonin'. It be said that the color of the sea be determined by many factors, such as the depth, the presence of certain organisms, and even the weather. \n\nWhen the sun be high in the sky, the sea often appears a shimmerin' shade of blue. This be because the water molecules be absorbin' most of the colors in the sunlight, 'cept for the blue wavelengths. So, what ye be seein' be the reflected blue light dancin' upon the surface of the water.\n\nBut if ye be sailin' into deeper waters, ye might find the sea takin' on a darker hue, like a rich navy blue or even black. This be because as ye descend into the depths, less light be able to penetrate the water, leavin' only the deepest blues to be seen.\n\nNow, there be times when the sea be not blue at all, but a mighty shade of brown. This 